In [2]:
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/yaicon/yaicon-maker-a.zip'  # ZIP 파일 경로
extract_folder = '/content/drive/MyDrive/yaicon/yaicon_maker_dataset'  # 압축 해제할 폴더

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/yaicon/yaicon-maker-a.zip'  # ZIP 파일 경로
extract_folder = '/content/drive/MyDrive/yaicon/yaicon_maker_dataset'  # 압축 해제할 폴더

## Set-up environment

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

In [ ]:
!pip install diffusers

## Define model

In [ ]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import zipfile


zip_path = '/content/drive/MyDrive/yaicon/yaicon-maker-a.zip'  # ZIP 파일 경로
extract_folder = '/content/drive/MyDrive/yaicon/yaicon_maker_dataset'  # 압축 해제 후 dataset 사진이 들어 있는 폴더 (fine-tuning 진행)

# 결과를 저장할 폴더 생성
output_folder = '/content/drive/MyDrive/yaicon/depthOutput'
os.makedirs(output_folder, exist_ok=True)
last_image_path = None  # 마지막 이미지의 경로를 저장할 변수 초기화

# 압축 해제된 폴더에서 이미지 처리
for filename in os.listdir(extract_folder):
    if filename.endswith('.jpg'):  # JPG 이미지만 처리
        image_path = os.path.join(extract_folder, filename)
        image = Image.open(image_path)

        pixel_values = feature_extractor(image, return_tensors="pt").pixel_values

        # Forward Pass
        with torch.no_grad():
            outputs = model(pixel_values)
            predicted_depth = outputs.predicted_depth

        # 예측된 깊이 맵 크기 조정
        prediction = torch.nn.functional.interpolate(
            predicted_depth.unsqueeze(1),
            size=image.size[::-1],
            mode='bicubic',
            align_corners=False
        ).squeeze().cpu().numpy()

        # 시각화 및 저장
        formatted = (prediction * 255 / np.max(prediction)).astype(np.uint8)
        depth_image = Image.fromarray(formatted)
        save_path = os.path.join(output_folder, f'depth_{filename}')
        depth_image.save(save_path)

        last_image_path = save_path  # 마지막으로 처리된 이미지의 경로를 업데이트

# 처리 완료 메시지와 마지막 이미지 표시
print("Processing complete.")
if last_image_path:
    last_image = Image.open(last_image_path)
    plt.imshow(last_image)
    plt.axis('off')  # 축 없애기
    plt.show()

In [ ]:
!pip install accelerate


In [ ]:
!pip install --upgrade diffusers huggingface_hub transformers


In [ ]:
import torch
from PIL import Image, ImageFilter
import numpy as np
from diffusers import StableDiffusionInpaintPipeline
from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer

# Diffusers 라이브러리를 사용하여 Stable Diffusion 인페인팅 파이프라인 로드
model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionInpaintPipeline.from_pretrained(model_id, variant="fp16", torch_dtype=torch.float16)
pipe = pipe.to(device)

# CLIP 모델을 사용한 특성 추출
feature_extractor = CLIPFeatureExtractor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
text_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32").to(device)


In [ ]:
from transformers import CLIPProcessor, CLIPModel
from diffusers import StableDiffusionInpaintPipeline

# CLIP 모델과 프로세서 초기화
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Stable Diffusion 인페인팅 모델 로드
inpaint_pipeline = StableDiffusionInpaintPipeline.from_pretrained("CompVis/stable-diffusion-v1-4-inpainting",
                                                                  revision="fp16", torch_dtype=torch.float16)
inpaint_pipeline = inpaint_pipeline.to("cuda")


In [ ]:
def inpaint_image(image_path, mask_path):
    # 이미지와 마스크 로드
    image = Image.open(image_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")  # 마스크는 흑백 이미지여야 합니다.

    # CLIP 모델을 사용하여 텍스트를 이미지와 관련된 벡터로 변환
    text_inputs = clip_processor(text=["A photo of a room with empty walls and no furniture."], return_tensors="pt", padding=True)
    text_embeddings = clip_model.get_text_features(**text_inputs.to("cuda"))

    # 인페인팅 파이프라인 실행
    inpainted_image = inpaint_pipeline(image=image, mask=mask, text_embeddings=text_embeddings,
                                       strength=0.75, guidance_scale=7.5)

    # 결과 이미지 반환
    return inpainted_image["images"][0]

# 예시 사용
image_path = "/path/to/your/image.jpg"
mask_path = "/path/to/your/mask.jpg"
result_image = inpaint_image(image_path, mask_path)
result_image.save("/path/to/save/inpainted_image.jpg")


In [ ]:
def get_inpating_mask(segmentation_mask: np.ndarray) -> Image:
    # ... 기존 함수 내용 유지 ...
    unique_colors = np.unique(segmentation_mask.reshape(-1, segmentation_mask.shape[2]), axis=0)
    unique_colors = [tuple(color) for color in unique_colors]
    segment_items = [map_colors_rgb(i) for i in unique_colors]

    control_items = ["windowpane;window", "wall", "floor;flooring","ceiling",  "sconce", "door;double;door", "light;light;source",
                     "painting;picture", "stairs;steps","escalator;moving;staircase;moving;stairway"]
    chosen_colors, segment_items = filter_items(
                colors_list=unique_colors,
                items_list=segment_items,
                items_to_remove=control_items
            )

    mask = np.zeros_like(segmentation_mask)
    for color in chosen_colors:
        color_matches = (segmentation_mask == color).all(axis=2)
        mask[color_matches] = 1

    mask_image = Image.fromarray((mask * 255).astype(np.uint8)).convert("RGB")
    # enlarge mask region so that it also will erase the neighborhood of masked stuff
    mask_image = mask_image.filter(ImageFilter.MaxFilter(25))
    return mask_image

def cleanup_room(image: Image, mask: Image) -> Image:
    # 텍스트를 CLIP 모델을 사용하여 토큰화 및 특성 추출
    text_inputs = tokenizer(["Empty room, with only empty walls, floor, ceiling, doors, windows"], return_tensors="pt", padding=True, truncation=True).to(device)
    text_features = text_model(**text_inputs).last_hidden_state

    # 이미지 및 마스크 사이즈 조정
    image = image.resize((512, 512))
    mask = mask.resize((512, 512))

    # 인페인팅 실행
    with torch.no_grad():
        image_inpainting = pipe(image=image, mask_image=mask, text_embeddings=text_features, guidance_scale=7.5).images[0]

    return image_inpainting

# 이미지와 마스크 로드
image = Image.open("path_to_your_image.jpg").convert("RGB")
mask = Image.open("path_to_your_mask.jpg").convert("RGB")

# 방 정리 실행
cleaned_room_image = cleanup_room(image, mask)
cleaned_room_image.save("path_to_save_cleaned_image.jpg")


# !!!!!여기서부터 !!!!!git clone

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pwd

'/home/work/roomMaker/reference/a_dataset_generator/get_empty_room'

In [8]:
!git clone https://github.com/Lavreniuk/generative-interior-design.git


fatal: destination path 'generative-interior-design' already exists and is not an empty directory.


In [3]:
!pip install diffusers

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.5 MB/s eta 0:00:00a 0:00:01
DEPRECATION: devscripts 2.22.1ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of devscripts or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [15]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: devscripts 2.22.1ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of devscripts or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [16]:
!pip install accelerate


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: devscripts 2.22.1ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of devscripts or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [14]:
!python get_empty_room.py

/usr/local/lib/python3.10/dist-packages/bitsandbytes-0.43.0.dev0-py3.10-linux-x86_64.egg/bitsandbytes/cuda_setup/main.py:122: UserWarning: 

BNB_CUDA_VERSION=XXX can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64
Loading: libbitsandbytes_cuda123.so


  warn(
False

===================================BUG REPORT===================================
/usr/local/lib/python3.10/dist-packages/bitsandbytes-0.43.0.dev0-py3.10-linux-x86_64.egg/bitsandbytes/cuda_setup/main.py:183: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
The following directories listed in your path were found to be non-exist

In [17]:
from getpass import getpass
import os

# Prompt for entering the token and set it as an environment variable
os.environ['HF_TOKEN'] = getpass('Enter your Hugging Face token here:')


In [18]:
import os
from PIL import Image
from tqdm import tqdm
import glob
# 올바른 모듈 임포트 방법 사용
from get_empty_room import get_segmentation_of_room, get_depth_image, get_inpating_mask, cleanup_room

def process_images():
    # 경로 설정
    save_dir = "/content/drive/MyDrive/yaicon/dataset_output/"
    image_paths = "/content/drive/MyDrive/yaicon/yaicon_maker_dataset/"
    os.makedirs(save_dir, exist_ok=True)

    # 이미지 처리
    for elem in tqdm(glob.glob(f"{image_paths}/*.*")):  # 모든 이미지 파일
        image_id = os.path.basename(elem).split(".")[0]
        image = Image.open(elem).convert("RGB")

        # 세그멘테이션
        color_map, segmentation_map = get_segmentation_of_room(image)
        inpainting_mask = get_inpating_mask(color_map)
        clean_room = cleanup_room(image, inpainting_mask)

        # 깊이 이미지 생성
        color_map_clean, segmentation_map_clean_room = get_segmentation_of_room(clean_room)
        depth_clean_room = get_depth_image(clean_room)

        # 파일 저장
        clean_room.save(os.path.join(save_dir, f"{image_id}_clean.png"))
        segmentation_map_clean_room.save(os.path.join(save_dir, f"{image_id}_segmentation.png"))
        depth_clean_room.save(os.path.join(save_dir, f"{image_id}_depth.png"))

# 함수 호출
process_images()



===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/extras/CUPTI/lib64'), PosixPath('/usr/local/nvidia/lib64')}
The following directories listed in your path were found to be non-existent: {PosixPath('7080/bai/ngc-pytorch'), PosixPath('24.01-pytorch2.2-py310-cuda12.3'), PosixPath('bai-repo')}
The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/lib/code-server-4.19.1/lib/vscode/remote/node_modules')}
The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {PosixPath('//proxy1.aitrain.ktcloud.com'), PosixPath('https'), PosixPath('10612/proxy/{{port}}')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
DEB

RuntimeError: Failed to import transformers.models.oneformer.modeling_oneformer because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [1]:
import torch
print(torch.version.cuda)


12.3


In [2]:
import os
from PIL import Image
from tqdm import tqdm
import glob
# 올바른 모듈 임포트 방법 사용
from get_empty_room import get_segmentation_of_room, get_depth_image, get_inpating_mask, cleanup_room

def process_images():
    # 경로 설정
    save_dir = "/home/work/roomMaker/reference/a_dataset_generator/get_empty_room/output"
    image_paths = "/home/work/roomMaker/dataset/airbnb"
    os.makedirs(save_dir, exist_ok=True)
    
    # 이미지 처리
    for elem in tqdm(glob.glob(f"{image_paths}/*.*")):  # 모든 이미지 파일
        image_id = os.path.basename(elem).split(".")[0]
        image = Image.open(elem).convert("RGB")
        
        # 세그멘테이션
        color_map, segmentation_map = get_segmentation_of_room(image)
        inpainting_mask = get_inpating_mask(color_map)
        clean_room = cleanup_room(image, inpainting_mask)
        
        # 깊이 이미지 생성
        color_map_clean, segmentation_map_clean_room = get_segmentation_of_room(clean_room)
        depth_clean_room = get_depth_image(clean_room)
        
        # 파일 저장
        clean_room.save(os.path.join(save_dir, f"{image_id}_clean.png"))
        segmentation_map_clean_room.save(os.path.join(save_dir, f"{image_id}_segmentation.png"))
        depth_clean_room.save(os.path.join(save_dir, f"{image_id}_depth.png"))

# 함수 호출
process_images()


This can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64

Could not find the bitsandbytes CUDA binary at PosixPath('/home/work/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda123.so')
Could not load bitsandbytes native library: /home/work/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: cannot open shared object file: No such file or directory
Traceback (most recent call last):
  File "/home/work/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py", line 109, in <module>
    lib = get_native_library()
  File "/home/work/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py", lin

AttributeError: module 'cv2.dnn' has no attribute 'DictValue'